# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
#pip install gmaps

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey
gmaps.configure(api_key=gkey)

In [3]:
#conda install -c conda-forge nodejs


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
mapsDF = pd.read_csv('../WeatherPy/cities.csv')
mapsDF

,city,lat,lon,temp,cloud,wind speed,humidity,country
0,hermanus,-34.4187,19.2345,8.22,6,3.55,82,ZA
1,busselton,-33.6500,115.3333,10.58,99,8.59,74,AU
2,victoria,22.2855,114.1577,29.49,78,3.13,89,HK
3,georgetown,5.4112,100.3354,26.61,40,1.54,87,MY
4,cherskiy,68.7500,161.3000,13.80,100,5.90,45,RU
...,...,...,...,...,...,...,...,...
568,nuuk,64.1835,-51.7216,3.34,90,5.14,87,GL
569,ulenje,-8.9333,33.6833,7.96,36,2.23,92,TZ
570,lasa,46.6166,10.7002,18.35,18,1.68,96,IT
571,iguape,-24.7081,-47.5553,17.56,67,3.18,84,BR


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
locations = mapsDF[["lat", "lon"]]

humidity = mapsDF['humidity']

In [6]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=30,
                                 point_radius=2)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
newDF = mapsDF.loc[(mapsDF["temp"] <26) & (mapsDF["temp"] >21) \
    & (mapsDF["wind speed"] <10) \
    & (mapsDF["cloud"] == 0)].dropna()

newDF

,city,lat,lon,temp,cloud,wind speed,humidity,country
56,marrakesh,31.6315,-8.0083,24.04,0,2.06,57,MA
64,brejo santo,-7.4933,-38.9872,21.24,0,2.83,67,BR
83,tiznit,29.5833,-9.5000,22.83,0,0.81,53,MA
84,gasa,31.5000,34.4667,23.34,0,0.61,74,PS
114,yerbogachen,61.2767,108.0108,25.92,0,3.03,44,RU
142,khash,28.2211,61.2158,23.44,0,0.63,54,IR
152,la ciotat,43.1748,5.6045,21.02,0,1.33,74,FR
198,nemea,37.8167,22.6667,23.69,0,1.18,60,GR
223,santa cruz,-17.8000,-63.1667,23.52,0,5.31,50,BO
234,mahon,39.8885,4.2658,23.31,0,1.54,88,ES


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = newDF.copy()
hotel_df["Hotel Name"] = ""
hotel_df

,city,lat,lon,temp,cloud,wind speed,humidity,country,Hotel Name
56,marrakesh,31.6315,-8.0083,24.04,0,2.06,57,MA,
64,brejo santo,-7.4933,-38.9872,21.24,0,2.83,67,BR,
83,tiznit,29.5833,-9.5000,22.83,0,0.81,53,MA,
84,gasa,31.5000,34.4667,23.34,0,0.61,74,PS,
114,yerbogachen,61.2767,108.0108,25.92,0,3.03,44,RU,
142,khash,28.2211,61.2158,23.44,0,0.63,54,IR,
152,la ciotat,43.1748,5.6045,21.02,0,1.33,74,FR,
198,nemea,37.8167,22.6667,23.69,0,1.18,60,GR,
223,santa cruz,-17.8000,-63.1667,23.52,0,5.31,50,BO,
234,mahon,39.8885,4.2658,23.31,0,1.54,88,ES,


In [9]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"type": "hotel",
          "keyword": "hotel",
          "radius": 5000,
          "key": gkey}

In [10]:
for index, row in hotel_df.iterrows():
    lat = row["lat"]
    lon = row["lon"]
    city_name = row["city"]
    

    params["location"] = f"{lat},{lon}"

    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(url, params=params).json()
    

    results = response["results"]

    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    
    print("------------")

print("-------End of Search--------")



Retrieving Results for Index 56: marrakesh.
Missing field/result... skipping.
------------
Retrieving Results for Index 64: brejo santo.
Missing field/result... skipping.
------------
Retrieving Results for Index 83: tiznit.
Missing field/result... skipping.
------------
Retrieving Results for Index 84: gasa.
Missing field/result... skipping.
------------
Retrieving Results for Index 114: yerbogachen.
Missing field/result... skipping.
------------
Retrieving Results for Index 142: khash.
Missing field/result... skipping.
------------
Retrieving Results for Index 152: la ciotat.
Missing field/result... skipping.
------------
Retrieving Results for Index 198: nemea.
Missing field/result... skipping.
------------
Retrieving Results for Index 223: santa cruz.
Missing field/result... skipping.
------------
Retrieving Results for Index 234: mahon.
Missing field/result... skipping.
------------
Retrieving Results for Index 242: petropavlovsk-kamchatskiy.
Missing field/result... skipping.
----

In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>city</dt><dd>{city}</dd>
<dt>country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

In [16]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content= hotel_info)

fig.add_layer(markers)


# save to PNG

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

In [18]:
plt.savefig("/Users/kylethomasmataloni/Desktop/python_api_challenge/VacactionPy/heatmap.png")
plt.show()

<Figure size 432x288 with 0 Axes>